# modernmt
> Library to use ModernMT machine translation services

In [ ]:
#| default_exp modernmt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| export
import requests, uuid
import os
from langcodes import *

class modernmttranslate:
    """
    Class to get translations from the ModernMT API
    """
    def __init__(self,adaptive=False,adaptation_tm=None,reference_tms=None):
        """Constructor of modernmttranslate class"""
        self._subscription_key = os.getenv('MMT_API_KEY')
        self._endpoint = "https://api.modernmt.com/"
        self._languages_cache = []
        self.adaptive = adaptive
        if adaptive:
            if adaptation_tm:
                self._adaptation_tm = adaptation_tm
            else:
                self._adaptation_tm = self.create_adaptation_tm()
                
    def create_adaptation_tm(self):
        """Function to create empty translation memory (TM) for adaptative MT.
        
        Returns
        -------
        int
            Returns ModernMT ID for created translation memory.
        
        """
        subscription_key = self._subscription_key
        endpoint = self._endpoint

        path = "memories"
        params = {
            'name': str(uuid.uuid4())
        }
        constructed_url = endpoint + path

        headers = {
            'X-HTTP-Method-Override': 'POST',
            'Content-Type' : 'application/json',
            'MMT-ApiKey' : self._subscription_key
        }

        request = requests.post(constructed_url, params=params, headers=headers)
        # TBD: error checking
        response = request.json()
        adaptation_tm_id = response["data"]["id"]
        
        return adaptation_tm_id
    
    def add_reference_translation(self, tuid, sourcelang, targetlang, source, reference):
        # TBD: should tuid be an integer or more universally a string?
        """Function to submit a new reference translation (most often post-edited translation) to the adaptation TM.
        
        Parameters
        ----------
        tuid: int
            Unique ID of the translation segment - if it exists, the segment gets overwritten
        sourcelang : str
            Source language identifier (BCP-47 format).
        targetlang : str
            Target language identifier (BCP-47 format).
        source: str
            Source text.
        reference: str
            Reference translation, possibly post-edited from machine translation.

        """
        subscription_key = self._subscription_key
        endpoint = self._endpoint

        path = "memories/"+str(self._adaptation_tm)+"/content"
        # Attention: using the same tuid as earlier will overwrite the TM entry
        params = {
            'tuid': tuid,
            'source': sourcelang,
            'target': targetlang,
            'sentence': source,
            'translation': reference
        }
        constructed_url = endpoint + path

        headers = {
            'X-HTTP-Method-Override': 'POST',
            'Content-Type' : 'application/json',
            'MMT-ApiKey' : self._subscription_key
        }

        request = requests.post(constructed_url, params=params, headers=headers)
        # TBD: error checking
        response = request.json()
        
        # Waiting for the import job is blocking - this is acceptable for evaluation, but probably not for live translation
        import_job = response["data"]["id"]
        finished = response["data"]["progress"]
        while not finished:
            path = "import-jobs/"+import_job
            params = {
            }
            constructed_url = endpoint + path

            headers = {
                'X-HTTP-Method-Override': 'GET',
                'Content-Type' : 'application/json',
                'MMT-ApiKey' : self._subscription_key
            }

            request = requests.post(constructed_url, params=params, headers=headers)
            # TBD: error checking
            response = request.json()
            finished = response["data"]["progress"]        
        
        return
        
    def check_langpair(self, sourcelang, targetlang):
        """Function to verify if a language pair identified by language ids is supported
        
        Parameters
        ----------
        sourcelang : str
            Source language identifier (BCP-47 format).
        targetlang : str
            Target language identifier (BCP-47 format).
            
        Returns
        -------
        bool
            Returns `True` if language pair is supported, otherwise `False`.
        
        """
        path = 'translate/languages'
        constructed_url = self._endpoint + path
        
        supported_languages = self._languages_cache
        # Cached language array empty if not initialized yet
        if not supported_languages:
            # Not passing API key here, because authentication isn't required for this API endpoint
            request = requests.get(constructed_url)
            response = request.json()
            supported_languages = response["data"]
            self._languages_cache = supported_languages
            
        language_pair_supported = False
        if tag_is_valid(sourcelang) and tag_is_valid(targetlang):
            if sourcelang in supported_languages and targetlang in supported_languages:
                language_pair_supported = True
        
        return language_pair_supported

    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language
        
        Parameters
        ----------
        sourcelang : str
            Source language identifier (BCP-47 format).
        targetlang : str
            Target language identifier (BCP-47 format).
        text : str
            Source text that is to be translated.
            
        Returns
        -------
        str
            Translated text.
        """
        translated_text = ""
        if self.check_langpair(sourcelang,targetlang):
            # Add your subscription key and endpoint
            subscription_key = self._subscription_key
            endpoint = self._endpoint

            path = "translate"
            params = {
                'source': sourcelang,
                'target': targetlang,
                'q': text
            }
            if self.adaptive:
                # TBD: Later need to add reference TMs too
                params['hints'] = str(self._adaptation_tm)
            constructed_url = endpoint + path

            headers = {
                'X-HTTP-Method-Override': 'GET',
                'Content-Type' : 'application/json',
                'MMT-ApiKey' : self._subscription_key
            }

            request = requests.post(constructed_url, params=params, headers=headers)
            response = request.json()
            translated_text = response["data"]["translation"]

        return translated_text

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
modernmt = modernmttranslate()
translation = modernmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una bella macchina!"

assert True == modernmt.check_langpair("en","it")
assert False == modernmt.check_langpair("en","gu")

### Testing adaptation

In [ ]:
#| hide
modernmt_adaptive = modernmttranslate(adaptive=True)
first_translation = modernmt_adaptive.translate_text("en","it","This is a cool car!")
assert first_translation == "Questa è una bella macchina!"
modernmt_adaptive.add_reference_translation(1,"en","it","This is a cool car!","Questa è una macchina fantastica!")
second_translation = modernmt_adaptive.translate_text("en","it","This is a cool car!")
assert second_translation == "Questa è una macchina fantastica!"

In [ ]:
show_doc(modernmttranslate.create_adaptation_tm)

In [ ]:
show_doc(modernmttranslate.add_reference_translation)

In [ ]:
show_doc(modernmttranslate.check_langpair)

In [ ]:
show_doc(modernmttranslate.translate_text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()